In [1]:
import pandas as pd
import os
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
from collections import Counter
import copy
from nameparser import HumanName


POSITION_BANK = ["President", "Chancellor", "Provost", "Director", "Dean", "Controller", "Trustee", "Member", "Regent", "Chairman", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Officer", "Chief", "Professor", "Commissioner", "Fellow", "Chairperson", "Manager", "Clergy", "Coordinator", "Auditor", "Governor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General"]
NON_BOARD_WORDS =["President", "Chancellor", "Provost", "Dean", "Controller", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Officer", "Chief", "Professor", "Commissioner", "Manager", "Clergy", "Coordinator", "Auditor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General", "Vice", "Chancellor,", "President,", "Executive", "Affairs", "Senior", "Associate", "Administration", "University", "College"]
BOARD_WORDS = ["Trustee", "Regent", "Member", "Fellow", "Overseer", "Governor", "Curator", "Visitor", "Manager"]
OTHER_BOARD_WORD = ["President", "Chairman", "Chairperson" ,"Treasurer", "Rector", "Member", "Secretary", "Ex Officio"]


In [2]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "2009"

path_read = f"{path_connected_data}{year}_split_positions.csv"
board_path = f"{path_temp_data}{year}_boards.csv"
second_board_path = f"{path_temp_data}{year}_double_boards.csv"
backup_board_path = f"{path_temp_data}{year}_backup_board.csv"
state_system_path = f"{path_temp_data}state_systems_validated.csv"

In [3]:
#Extract the names of all the institutions
def extract_institutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["Institution"] not in institution_list:
            institution_list.append(row["Institution"])
    print(institution_list)
    return institution_list

#determine what each universities board is called by counting the most common words in the position column, and picking only those who are valid board names.
#Also determine if there are multiple boards
def determine_board_position(df):
    most_frequent = {}
    two_boards = {}
    grouped_df = df.groupby("Institution")
    for key, value in grouped_df:
        word_count = Counter()
        for position in value["Position"]:
            if not pd.isna(position):
                individual_words = position.split()
            else:
                individual_words = ""
            filtered_words = [word for word in individual_words if word in BOARD_WORDS]
            word_count.update(filtered_words)
        if word_count:
            common_words = word_count.most_common(2)
            most_frequent[key] = common_words[0][0]
            #if there are two board words that come up, and the second board word comes up frequently, likely is a school with two boards
            if len(common_words) >= 2:
                large_enough_board = common_words[1][1] >= (common_words[0][1] / 4) or (common_words[0][1] >= 8)
            else:
                large_enough_board = False
            if len(common_words) >= 2 and large_enough_board and common_words[1][0] != "Director":
                if common_words[1][0].strip().lower() != common_words[0][0].strip().lower():
                    two_boards[key] = common_words[1][0]
            else:
                two_boards[key] = None
        else:
            most_frequent[key] = None
            two_boards[key] = None
    return most_frequent, two_boards


def determine_director_schools(df):
    most_frequent = {}
    two_boards = {}
    grouped_df = df.groupby("Institution")
    for key, value in grouped_df:
        word_count = Counter()
        for position in value["Position"]:
            individual_words = position.split()
            board_words_dir = ['Director']
            filtered_words = [word for word in individual_words if word in board_words_dir]
            word_count.update(filtered_words)
        if word_count:
            common_words = word_count.most_common(2)
            most_frequent[key] = common_words[0][0]
            #if there are two board words that come up, and the second board word comes up frequently, likely is a school with two boards
            if len(common_words) >= 2:
                large_enough_board = common_words[1][1] >= (common_words[0][1] / 4) or (common_words[0][1] >= 8)
            else:
                large_enough_board = False
            if len(common_words) >= 2 and large_enough_board:
                if common_words[1][0].strip().lower() != common_words[0][0].strip().lower():
                    two_boards[key] = common_words[1][0]
            else:
                two_boards[key] = None
        else:
            most_frequent[key] = None
            two_boards[key] = None
    return most_frequent, two_boards

In [4]:
#board position titles for each institution:
full_df = pd.read_csv(path_read)
for index, row in full_df.iterrows():
    if pd.isna(row["Position"]):
        full_df.at[index, "Position"] = "nan"
university_boards, double_boards = determine_board_position(full_df)
for key, value in university_boards.items():
    print(key,": ",  value)

Adelphi University :  Trustee
Agnes Scott College :  Trustee
Albion College :  Trustee
Alcorn State University :  None
Alfred University :  Trustee
Allegheny College :  Trustee
Alliant International University :  Trustee
American University :  Trustee
Amherst College :  Trustee
Andrews University :  Trustee
Angelo State University :  None
Appalachian State University :  None
Aquinas College :  Trustee
Arcadia University :  Trustee
Arizona State University :  Regent
Assumption College :  Trustee
Auburn University :  Trustee
Auburn University At Montgomery :  None
Augusta State University :  None
Austin Peay State University :  Regent
Baker University :  Trustee
Baldwin Wallace College :  Trustee
Ball State University :  Trustee
Bard College :  Trustee
Barnard College :  Trustee
Barry University :  Trustee
Bates College :  Trustee
Baylor College Of Medicine :  Member
Baylor University :  Regent
Belmont University :  Trustee
Beloit College :  Trustee
Bernard M Baruch College :  None
Biola

In [5]:
#helper functions
def higher_ascii(char1, char2):    
    if ord(char1.upper()) >= ord(char2.upper()):
        return True
    else:
        return False

def parse_name(raw_name):
    raw = raw_name.replace("Rev", "")
    raw = raw.replace("Very ", "")
    parsed_name = HumanName(raw)
    parsed_name.suffix = ""
    pre_parse = str(parsed_name)
    split_name = pre_parse.split(" ")
    last_name = parsed_name.last
    if last_name == "" or " " in last_name or last_name == '.':
        return split_name[-1]
    else:
        return str(last_name)

In [6]:
#grouping + expansion

#determine where each board starts and ends in the dataframe (indices)
def find_word_grouping(df, board_name):
    grouped_df = df.groupby("Institution")
    first_board_occurrence, last_board_occurrence, first_institution_occurrence = {}, {}, {}
    result_dataframe = []
    for key, value in grouped_df:
        if key in board_name:        
            board_position = board_name[key]
        else:
            board_position = None
        first_institution_occurrence[key] = value.index[0]
        if board_position is not None:
            all_members = value["Position"].tolist()
            try:
                #experiement between "in" and ==
                first_index = next(i for i, pos in enumerate(all_members) if board_position == pos.title())
                last_index = len(all_members) - next(i for i, pos in enumerate(reversed(all_members)) if board_position in pos.title()) - 1
                if first_index == None:
                    last_index = None
                
                first_board_occurrence[key] = value.index[first_index]
                last_board_occurrence[key] = value.index[last_index]
                result_dataframe.append(value.iloc[first_index:last_index+1])
            except StopIteration:
                first_board_occurrence[key] = value.index[0]
                last_board_occurrence[key] = value.index[-1]
                # continue
        else:
            first_board_occurrence[key] = value.index[0]
            last_board_occurrence[key] = value.index[-1]
    return pd.concat(result_dataframe), first_board_occurrence, last_board_occurrence, first_institution_occurrence

def find_word_grouping_substring(df, board_name):
    grouped_df = df.groupby("Institution")
    first_board_occurrence, last_board_occurrence, first_institution_occurrence = {}, {}, {}
    result_dataframe = []
    for key, value in grouped_df:
        if key in board_name:
            board_position = board_name[key]
        else:
            board_position = None
        first_institution_occurrence[key] = value.index[0]
        if board_position is not None:
            all_members = value["Position"].tolist()
            try:
                #experiement between "in" and ==
                first_index = next(i for i, pos in enumerate(all_members) if board_position in pos.title())
                last_index = len(all_members) - next(i for i, pos in enumerate(reversed(all_members)) if board_position in pos.title()) - 1 
                first_board_occurrence[key] = value.index[first_index]
                last_board_occurrence[key] = value.index[last_index]
                result_dataframe.append(value.iloc[first_index:last_index+1])
            except StopIteration:
                first_board_occurrence[key] = value.index[0]
                last_board_occurrence[key] = value.index[-1]
                # continue
        else:
            first_board_occurrence[key] = value.index[0]
            last_board_occurrence[key] = value.index[-1]
    return pd.concat(result_dataframe), first_board_occurrence, last_board_occurrence, first_institution_occurrence

#verify alphabetical ordering of names when expanding boards upwards
def verify_ordering(full_df, current_board_start, count):
    original_board_position = full_df.iloc[current_board_start]
    original_last_name = parse_name(original_board_position["Name"])
    while count > 0:
        current_position = full_df.iloc[current_board_start - count]
        current_last_name = parse_name(current_position["Name"])
        correct_ordering = higher_ascii(original_last_name[0], current_last_name[0])
        if not correct_ordering:
            return False
        count-=1
    return True

def expand_single_board_upward(full_df, grouped_boards, board_indices_start):
    sorted_keys = sorted(board_indices_start.keys(), key=lambda k: board_indices_start[k])
    for key in sorted_keys:
        expanded_flag = False
        #arbitrary value for amount of rows we can expand a board upwards
        count = 4
        current_board_start = board_indices_start[key]
        while count > 0:
            previous_df_row = full_df.iloc[current_board_start - count]
            previous_row_position = previous_df_row["Position"].title()
            original_df_row  = full_df.iloc[current_board_start]
            original_row_position = original_df_row["Position"].title()
            same_institution = original_df_row["Institution"] == key
            board_position = any(p in previous_row_position for p in OTHER_BOARD_WORD)
            if same_institution and (board_position or expanded_flag):
                #check for alphabetical orderings of last names
                if not expanded_flag:
                    alphabetical_order = verify_ordering(full_df, current_board_start, count)
                if (alphabetical_order and "Dean" not in previous_row_position) or (expanded_flag):
                    # print("expanding upward: ", key)
                    grouped_boards[key] = pd.concat([pd.DataFrame([previous_df_row]), grouped_boards.get(key, pd.DataFrame())])
                    expanded_flag = True
            count-=1
    return grouped_boards


def expand_double_board_upward(full_df, grouped_boards, board_indices_start, double_boards):
    sorted_keys = sorted(board_indices_start.keys(), key=lambda k: board_indices_start[k])
    for key in sorted_keys:
        expanded_flag = False
        #arbitrary value for amount of rows we can expand a board upwards
        count = 4
        current_board_start = board_indices_start[key]
        while count > 0:
            previous_df_row = full_df.iloc[current_board_start - count]
            previous_row_position = previous_df_row["Position"].title()
            original_df_row  = full_df.iloc[current_board_start]
            same_institution = original_df_row["Institution"] == key
            board_position = any(p in previous_row_position for p in OTHER_BOARD_WORD)
            if same_institution and (board_position or expanded_flag) and double_boards[key] is not None:
                #check for alphabetical orderings of last names
                if not expanded_flag:
                    alphabetical_order = verify_ordering(full_df, current_board_start, count)
                if (alphabetical_order and "Dean" not in previous_row_position) or (expanded_flag):
                    # print("expanding upward: ", key)
                    grouped_boards[key] = pd.concat([pd.DataFrame([previous_df_row]), grouped_boards.get(key, pd.DataFrame())])
                    expanded_flag = True
            count-=1
    return grouped_boards


#expand board downwards
def expand_single_board_downward(full_df, grouped_boards, board_indices_end, first_institution_index): 
    sorted_keys = sorted(board_indices_end.keys(), key=lambda k: board_indices_end[k])
    for key in sorted_keys:
        if sorted_keys.index(key) < len(sorted_keys) - 1:
            current_board_end = board_indices_end[key]
            next_key = sorted_keys[sorted_keys.index(key) + 1]
            next_board_start = first_institution_index[next_key]
            if (current_board_end + 3 >= next_board_start and current_board_end + 1 != next_board_start):
                for index in range(current_board_end + 1, next_board_start):
                    # print("expanding downard: ", key)
                    grouped_boards[key] = pd.concat([grouped_boards.get(key, pd.DataFrame()), pd.DataFrame([full_df.iloc[index]])])
    return grouped_boards

#expand board downards for secondary boards
def expand_double_board_downward(full_df, grouped_boards, double_board_indices_start, double_board_indices_end, first_institution_index, first_board_indices_start, first_board_indices_end, double_boards):
    sorted_keys = sorted(double_board_indices_end.keys(), key=lambda k: double_board_indices_end[k])
    for key in sorted_keys:
        if sorted_keys.index(key) < len(sorted_keys) - 1 and double_boards[key] is not None:
            current_board_end = double_board_indices_end[key]
            next_key = sorted_keys[sorted_keys.index(key) + 1]
            next_board_start = first_institution_index[next_key]
            current_board_start = double_board_indices_start[key]
            first_board_end = first_board_indices_end[key]
            first_board_start = first_board_indices_start[key]
            if (current_board_start > first_board_end) and (current_board_end + 3 >= next_board_start and current_board_end + 1 != next_board_start):
                for index in range(current_board_end + 1, next_board_start):
                    # print("expanding double board downward: ", key)
                    grouped_boards[key] = pd.concat([grouped_boards.get(key, pd.DataFrame()), pd.DataFrame([full_df.iloc[index]])])
            elif current_board_start < first_board_end and (current_board_end + 3 >= next_board_start and current_board_end + 1 != next_board_start):
                for index in range(current_board_end + 1, first_board_start):
                    # print("expanding double board downward: ", key)
                    grouped_boards[key] = pd.concat([grouped_boards.get(key, pd.DataFrame()), pd.DataFrame([full_df.iloc[index]])])
    return grouped_boards

#expand board downwards
def expand_director_board_downward(full_df, grouped_boards, board_indices_end, first_institution_index): 
    sorted_keys = sorted(board_indices_end.keys(), key=lambda k: board_indices_end[k])
    for key in sorted_keys:
        if sorted_keys.index(key) < len(sorted_keys) - 1:
            current_board_end = board_indices_end[key]
            next_key = sorted_keys[sorted_keys.index(key) + 1]
            next_board_start = first_institution_index[next_key]
            # if (current_board_end + 3 >= next_board_start and current_board_end + 1 != next_board_start):
            if current_board_end + 3 < next_board_start:
                next_board_start = current_board_end+3
            for index in range(current_board_end + 1, next_board_start):
                # print("expanding downard: ", key)
                grouped_boards[key] = pd.concat([grouped_boards.get(key, pd.DataFrame()), pd.DataFrame([full_df.iloc[index]])])
                index+=1
    return grouped_boards

In [7]:
def expand_board(full_df, board_df, board_indices_start, board_indices_end, first_institution_index):
    initial_boards = list(set(board_df["Institution"].values))
    grouped_boards = {name: group for name, group in board_df.groupby("Institution")}
    grouped_boards = expand_single_board_upward(full_df, grouped_boards, board_indices_start)
    grouped_boards = expand_single_board_downward(full_df, grouped_boards, board_indices_end, first_institution_index)  
    combined_boards = pd.concat(grouped_boards.values())
    final_boards = list(set(combined_boards["Institution"].values))
    indices_to_drop = []
    # print(len(initial_boards))
    # print(len(final_boards))
    for index, row in combined_boards.iterrows():
        inst = row["Institution"]
        if inst in final_boards and inst not in initial_boards:
            indices_to_drop.append(index)
    cleaned__df = combined_boards.drop(index=indices_to_drop).reset_index(drop=True)
    # cleaned__df = combined_boards.drop(index=indices_to_drop)
    return cleaned__df

def expand_double_board(full_df, board_df, board_indices_start, board_indices_end, first_institution_index, double_boards, first_board_indices_start, first_board_indices_end):
    grouped_boards = {name: group for name, group in board_df.groupby("Institution")}
    grouped_boards = expand_double_board_upward(full_df, grouped_boards, board_indices_start, double_boards)
    grouped_boards = expand_double_board_downward(full_df, grouped_boards, board_indices_start, board_indices_end, first_institution_index, first_board_indices_start, first_board_indices_end, double_boards)
    combined_boards = pd.concat(grouped_boards.values())
    return combined_boards

def expand_directors(full_df, board_df, board_indices_start, board_indices_end, first_institution_index):
    grouped_boards = {name: group for name, group in board_df.groupby("Institution")}
    grouped_boards = expand_single_board_upward(full_df, grouped_boards, board_indices_start)
    grouped_boards = expand_director_board_downward(full_df, grouped_boards, board_indices_end, first_institution_index)  
    combined_boards = pd.concat(grouped_boards.values())
    return combined_boards

def assemble_board_dict(board_df):
    board_dict = {}
    for institution in board_df['Institution'].unique():
        rows = board_df[board_df["Institution"] == institution]
        board_dict[institution] = rows
    return board_dict

In [8]:
#cleaning + deletion

def clean_false_members(expanded_boards, university_boards, original_boards):
    indices_to_drop = []
    for index, row in expanded_boards.iterrows():
        pos = row["Position"]
        if "Dean" in pos or "Director" in pos:
            indices_to_drop.append(index)
    cleaned__df = expanded_boards.drop(index=indices_to_drop).reset_index(drop=True)
    # cleaned__df = expanded_boards.drop(index=indices_to_drop)
    return cleaned__df
            
def delete_overlap(primary_boards, secondary_boards):
    for index, row in secondary_boards.iterrows():
        if any(row.equals(primary_row) for _, primary_row in primary_boards.iterrows()):
            secondary_boards.drop(index, inplace=True)
    return secondary_boards

def validate_double_boards(board_dict, double_boards):
    invalid_list = []
    for index, row in double_boards.iterrows():
        institution = row["Institution"]
        if institution in board_dict:
            # Check if the row exists in the DataFrame for the institution
            original_board = board_dict[institution]
            # Convert the row to a DataFrame and compare
            is_row_in_df = original_board.apply(lambda x: x.equals(row), axis=1).any()
            if is_row_in_df and institution not in invalid_list:
                print(f"Invalid board: {institution}")
                invalid_list.append(institution)
    return invalid_list

def verify_ordering_entire_board(full_df):
    # Initialize a set to store indices of rows to remove
    indices_to_remove = set()
    # Group by 'Institution' and check the ordering
    for key, group in full_df.groupby('Institution'):
        if len(group) <= 4:
            indices_to_remove.update(group.index)
            continue
        count = 0
        previous_last_name = "000"
        for i, row in group.iterrows():
            name = row["Name"]
            current_last_name = parse_name(name)
            correct_ordering = higher_ascii(current_last_name[0], previous_last_name[0])
            if not correct_ordering and count <= 2:
                print(key, name, "  ", previous_last_name)
                # Mark the previous row for removal
                indices_to_remove.add(i - 1)
            count += 1
            previous_last_name = current_last_name
    print("here ", indices_to_remove)
    modified_df = full_df.drop(index=list(indices_to_remove))
    modified_df.reset_index(drop=True, inplace=True)
    return modified_df

def clean_false_members_directors(df):
    indices_to_drop = []
    for index, row in df.iterrows():
        pos = row["Position"]
        if "Dean" in pos or "Director," in pos:
            indices_to_drop.append(index)
    cleaned__df = df.drop(index=indices_to_drop).reset_index(drop=False)
    return cleaned__df

In [9]:
id_dict = {}
for index, row in full_df.iterrows():
    id_dict[row["Institution"]] = row["AffiliationId"]

#mark state system boards 
state_systems = pd.read_csv(state_system_path)
system_id_dict = {}
system_inst_dict = {}
for index, row in state_systems.iterrows():
    if not pd.isna(row["StateSystem"]):
        system_id_dict[row["AffiliationId"]] = row["StateSystem"] 
        system_inst_dict[row["Institution"]] = row["StateSystem"]

#remove (dash, comma, period from system inst dict for better matching)
system_inst_dict_cleaned = {
    key.replace("-", " ").replace(",", "").replace(".", ""): value
    for key, value in system_inst_dict.items()
}

In [10]:
#Contains largest board at each insititution (mostly schools have 1 but for the schools with 2, the larger one is chosen)
institution_df = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
original_single_boards, single_board_indices_start, single_board_indices_end, single_first_institution_index = find_word_grouping(full_df, university_boards)
full_first_board = expand_board(full_df, original_single_boards, single_board_indices_start, single_board_indices_end, single_first_institution_index)
full_first_board = clean_false_members(full_first_board, university_boards, original_single_boards)


In [11]:
#schools with a board, but board position doesnt appear as exact string (e.g. Member -> "Board Member")
unmarked_boards = {}
for key, value in university_boards.items():
    if value != None and key not in full_first_board["Institution"].values:
        unmarked_boards[key] = value
        print(key + ": ", value)

Auburn University:  Trustee
Baylor College Of Medicine:  Member
Birmingham Southern College:  Member
California State University San Bernardino:  Manager
Citadel:  Member
Clemson University:  Member
College Of William And Mary:  Member
Colorado State University System:  Member
Dickinson College:  Manager
Emory University:  Trustee
Frostburg State University:  Manager
George Mason University:  Member
Georgia State University:  Manager
Grand Valley State University:  Member
Idaho State University:  Member
Indiana University Kokomo:  Manager
Indiana University System:  Trustee
James Madison University:  Member
Lake Forest College:  Trustee
Longwood University:  Member
Michigan Technological University:  Member
Minnesota State University Mankato:  Trustee
Minnesota State University Moorhead:  Trustee
Montana Tech Of University Of Montana:  Manager
North Dakota University System:  Member
Old Dominion University:  Member
Oregon University System:  Member
Pennsylvania State University:  Trust

In [12]:
#Schools with two boards
original_double_boards, double_board_indices_start, double_board_indices_end, double_first_institution_index = find_word_grouping(full_df, double_boards)
full_second_board = expand_double_board(full_df, original_double_boards, double_board_indices_start, double_board_indices_end, double_first_institution_index, double_boards, single_board_indices_start, single_board_indices_end)
full_second_board = clean_false_members(full_second_board, double_boards, original_double_boards)

#schools that have two boards
# for key, value in double_boards.items():
#     if value is not None:
#         print(key, ": ", value)

In [13]:
#create initial board using substring position matching instead of exact matches
substring_boards, substring_board_indices_start, substring_board_indices_end, substring_first_institution_index = find_word_grouping_substring(full_df, unmarked_boards)
full_substring_board = expand_board(full_df, substring_boards, substring_board_indices_start, substring_board_indices_end, substring_first_institution_index)
full_substring_board = clean_false_members(full_substring_board, unmarked_boards, substring_boards)

#remove members based on alphabetical ordering of last names, remove names who didn't appear in order, then recreate df
validated_substring_board = verify_ordering_entire_board(full_substring_board)
validated_substring_df, validated_substring_indices_start, validated_substring_indices_end, validated_substring_first_inst_index = find_word_grouping_substring(validated_substring_board, unmarked_boards)
validated_substring_board = expand_board(validated_substring_board, validated_substring_df, validated_substring_indices_start, validated_substring_indices_end, validated_substring_first_inst_index)
validated_substring_board = clean_false_members(validated_substring_board, unmarked_boards, validated_substring_df)

Auburn University Charles W. Bruce    Davis
Auburn University Lee F. Armstrong    Bruce
Citadel Col Glenn D. Addison    Baiden
University Of Mary Washington Randall R. Elev    Moss
Winthrop University Anthony J. DiGiorgio    Faust
here  {0, 1, 262, 523, 651, 183, 73, 74, 224, 225, 608, 609, 610, 739, 619, 244, 373, 631, 505, 506, 507}


In [14]:
director_common, double_directors = determine_director_schools(full_df)
director_inst_boards = {}
for key, value in director_common.items():
    if university_boards[key] == None and value != None:
        # print(key)
        director_inst_boards[key] = value
institutions_to_keep = list(director_inst_boards.keys())
director_df = full_df[full_df['Institution'].isin(institutions_to_keep)]


In [15]:
#create initial board using substring position matching instead of exact matches
director_boards, director_board_indices_start, director_board_indices_end, director_first_institution_index = find_word_grouping(director_df, director_inst_boards)
full_director_board = expand_directors(full_df, director_boards, director_board_indices_start, director_board_indices_end, director_first_institution_index)
full_director_board = clean_false_members_directors(full_director_board)

validated_director_board = verify_ordering_entire_board(full_director_board)
validated_director_df, removed_director_indices_start, removed_director_indices_end, removed_director_institution_index = find_word_grouping(validated_director_board, director_inst_boards)
final_director_board = expand_directors(validated_director_board, validated_director_df, removed_director_indices_start, removed_director_indices_end, removed_director_institution_index)
final_director_board = clean_false_members_directors(final_director_board)

Creighton University Rev. Edwin H. "Ned" Cassem, S.J    Bay
University Of Montana Western , Business Services (Vacant)    000
here  {0, 37, 38, 66, 67, 111, 133, 134, 181, 182, 219, 220, 221, 222, 231, 237, 238, 239, 240, 241, 242, 249, 260, 261, 262, 263, 272, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 359, 360, 361, 362, 363, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472}


In [16]:
grouped_dict = {key: value for key, value in final_director_board.groupby('Institution')}
keys_to_remove = [institution for institution, group in grouped_dict.items() if group['Position'].str.contains('Director,', case=False).any()]
# Remove the keys from the dictionary
for key in keys_to_remove:
    del grouped_dict[key]
final_director_board = pd.concat(grouped_dict.values()).reset_index(drop=True)

insts = final_director_board["Institution"].values
for x in insts:
    university_boards[x] = "Director"

In [17]:
#combine director, single, and substring board together
combined_single_boards = pd.concat([full_first_board, validated_substring_board, final_director_board], ignore_index = True)

grouped = combined_single_boards.groupby("Institution")
filtered_groups = {institution: group for institution, group in grouped if len(group) >= 4}
filtered_combined_single_boards = pd.concat(filtered_groups.values()).reset_index(drop=True)

filtered_combined_single_boards = filtered_combined_single_boards.sort_values(by="Institution")
# filtered_combined_single_boards.to_csv(board_path, index = False)

In [18]:
#delete boards from the double board df which are just subsets of the single boards
single_board_copy = filtered_combined_single_boards.copy(deep=True)
board_dict = assemble_board_dict(single_board_copy)
invalid_double_boards = validate_double_boards(board_dict, full_second_board)
for index, row in full_second_board.iterrows():
    if row["Institution"] in invalid_double_boards:
        full_second_board.drop(index, inplace=True)
# full_second_board.to_csv(second_board_path, index = False)

In [19]:
filtered_combined_single_boards["StateSystem"] = ""
id_name_dict = {}
inst_name_list = []
missing_institutions = []
#mark state systems
for index, row in filtered_combined_single_boards.iterrows():
    if row["AffiliationId"] in system_id_dict:
        filtered_combined_single_boards.at[index, "StateSystem"] = system_id_dict[row["AffiliationId"]]
#add all ids and names
for index, row in full_df.iterrows():
    inst = row["Institution"]
    inst = inst.replace("-", " ").replace(",", "").replace(".", "")
    id_name_dict[inst] = row["AffiliationId"]
    inst_name_list.append(inst)

all_board_ids = list(set(np.concatenate((filtered_combined_single_boards["AffiliationId"].values,  full_second_board["AffiliationId"].values))))

all_board_names = list(set(np.concatenate((filtered_combined_single_boards["Institution"].values, full_second_board["Institution"].values))))
all_board_names_cleaned = []
for name in all_board_names:
    name_cleaned = name.replace("-", " ").replace(",", "").replace(".", "")
    all_board_names_cleaned.append(name_cleaned)

for inst, id in id_name_dict.items():
    if id not in all_board_ids and inst not in all_board_names_cleaned and id not in system_id_dict and inst not in system_inst_dict_cleaned:
        missing_institutions.append(inst)

for x in missing_institutions:
    print(x)


Auburn University At Montgomery
Chestnut Hill College
Colorado State University
Colorado State University Pueblo
Concordia University
Dickinson College
Drake University
Duke University Health System
Elmira College
Furman University
Hampton University
Hardin Simmons University
Lynchburg College In Virginia
Mississippi Institutions Of Higher Learning
Montana State University Northern
Ohio Wesleyan University
Pacific Lutheran University
Indiana University Purdue University Fort Wayne
Purdue University Calumet
Purdue University North Central
St Johns College
St Marys University Texas
Samford University
Lamar Institute Of Technology
Tulane University
University Of Denver
University Of Michigan Dearborn
University Of Michigan Flint
University Of Rochester
University Of Texas Pan American
Medical College Of Georgia
Washington University Medical Center
Wittenberg University


In [20]:
print(system_inst_dict_cleaned)

{'Arizona State University': 'Arizona Board of Regents', 'Austin Peay State University': 'Tennessee Board of Regents', 'Austin Pay State University': 'Tennessee Board of Regents', 'Bowling Green State University': 'University System of Ohio', 'California Maritime Academy': 'California State University System', 'California Maritime Academy [Cal Maritime]': 'California State University System', 'California Polytechnic State University': 'California State University System', 'California Polytechnic State University San Luis Obispo': 'California State University System', 'California Polytechnic State University San Luis Obispo [Cal Poly]': 'California State University System', 'California State Polytechnic University': 'California State University System', 'California State Polytechnic University Pomona': 'California State University System', 'California State University Los Angeles': 'California State University System', 'California State University Bakersfield': 'California State Univers

In [21]:
position_counts = Counter(filtered_combined_single_boards['Position'])

sorted_position_counts = dict(sorted(position_counts.items(), key=lambda item: item[1], reverse=True))
for position, count in sorted_position_counts.items():
    print(position, ":", count)

Trustee : 7661
Regent : 921
Vice Chairman : 358
Chairman : 342
Board Member : 294
Director : 271
Secretary : 182
Ex Officio Trustee : 170
Member : 168
Overseer : 96
Governor : 86
Executive Committee Member : 75
Charter Trustee : 75
Alumni Trustee : 73
Student Trustee : 70
Treasurer : 66
Manager : 64
Ex Officio Regent : 39
Student Regent : 39
Public Trustee : 33
President : 31
Board-Elected Trustee : 26
Board Elected Trustee : 24
Faculty Trustee : 23
Gubernatorial Appointed Trustee : 22
Trustee Emeritus : 21
Vice President : 21
Regent-at-Large : 19
Faculty Representative : 18
Assistant Secretary : 17
Alumni Elected Trustee : 17
Trusted : 16
National Trustee : 16
Fellow : 15
Chairman Emeritus : 14
Senior Trustee : 13
Student Representative : 12
Vacant : 12
Governor Appointed Trustee : 11
Rector : 11
Commonwealth Trustee : 11
Alumni Association Nominated Trustee : 10
Vice Rector : 10
Term Trustee : 10
Faculty Regent : 9
Chair : 9
Special Representative to the Board : 9
Ex Officio Board Me

In [22]:
#mark board members, presidents, chairmans, etc
#currently not marking student reps as normal members
CHAIRPERSONS = ["Chairman", "Chairperson", "President", "Chair", "Chancellor"]
MEMBERS = ["Trustee", "Regent", "Member", "Fellow", "Overseer", "Governor", "Curator", "Visitor", "Manager", "Director"]
OTHER_BOARD_WORD = ["Treasurer", "Faculty Representative", "Rector", "Secretary", "Counsel", "Clerk", "Vacant", "Executive Committee Member", "Special", "Student", "Chief Executive Officer", "Affiliation", "Justice", "Registrar", "Staff Representative", "Librarian",
                    "Alumni Representative", "Faculty Visitor", "Chief Investment Officer"]

def mark_members(board_df):
    board_df["FixedPosition"] = ""
    grouped_boards = board_df.groupby("Institution")
    for key, value in grouped_boards:
        for index, row in value.iterrows():
            position = row["Position"].title()
            board_name = university_boards[key]
            pres_appears = any(pos in position for pos in CHAIRPERSONS)
            if board_name is None:
                board_name = "zZz01" 

            if board_name in row["Position"]:
                board_df.at[index, "FixedPosition"] = board_name
            elif  pres_appears and "Vice" not in position:
                board_df.at[index, "FixedPosition"] = "Board President"
            elif pres_appears and "Vice" in position:
                board_df.at[index, "FixedPosition"] = "Board Vice President"
            elif any(pos in position for pos in OTHER_BOARD_WORD):
                board_df.at[index, "FixedPosition"] = "Other Board Member"
            else:
                board_df.at[index, "FixedPosition"] = board_name

            if "Ex Officio" in row["Position"]:
                board_df.at[index, "FixedPosition"] += ", Ex Officio"
    return board_df


In [23]:
marked_boards_single_df = mark_members(filtered_combined_single_boards)
marked_boards_single_df = marked_boards_single_df.drop_duplicates(keep = False)
marked_boards_single_df.to_csv(board_path, index = False)

In [24]:
marked_boards_double_df = mark_members(full_second_board)
marked_boards_double_df = marked_boards_double_df.drop_duplicates(keep =False)
marked_boards_double_df.to_csv(second_board_path, index = False)

In [25]:
for index, row in marked_boards_single_df.iterrows():
    if pd.isna(row["FixedPosition"]) or row["FixedPosition"] == "":
        print(row["Position"])